In [ ]:
# default_exp inference

# Inference

> This contains the code required for inference.

In [ ]:
# export
from fastai.learner import load_learner
from fastai.callback.core import GatherPredsCallback
from fastai.learner import Learner
from fastcore.basics import patch
from fastcore.meta import delegates

In [ ]:
#export
@patch
def get_X_preds(self: Learner, X, y=None, bs=64, with_input=False, with_decoded=True, with_loss=False):
    if with_loss and y is None:
        print("cannot find loss as y=None")
        with_loss = False
    dl = self.dls.valid.new_dl(X, y=y, bs=bs)
    output = list(self.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss, reorder=False))
    if with_decoded and len(self.dls.tls) >= 2 and hasattr(self.dls.tls[-1], "tfms") and hasattr(self.dls.tls[-1].tfms, "decodes"):
        output[2 + with_input] = self.dls.tls[-1].tfms.decode(output[2 + with_input])
    return tuple(output)

Get the predictions and targets, optionally with_input and with_loss.

with_decoded will also return the decoded predictions (it reverses the transforms applied).

The order of the output is the following: 

- input (optional): if with_input is True
- **probabiblities** (for classification) or **predictions** (for regression)
- **target**: if y is provided. Otherwise None.
- **predictions**: predicted labels. Predictions will be decoded if with_decoded=True.
- loss (optional): if with_loss is set to True and y is not None.

In [ ]:
from tsai.data.external import get_UCR_data
dsid = 'OliveOil'
X, y, splits = get_UCR_data(dsid, split_data=False)
X_test = X[splits[1]]
y_test = y[splits[1]]

In [ ]:
learn = load_learner("./models/test.pth")

⚠️ Warning: load_learner (from fastai) requires all your custom code be in the exact same place as when exporting your Learner (the main script, or the module you imported it from).

In [ ]:
test_probas, test_targets, test_preds = learn.get_X_preds(X_test, with_decoded=True)
test_probas, test_targets, test_preds

(TSTensor(vars:30, len:4, device=cpu),
 None,
 array(['4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4'], dtype='<U1'))

In [ ]:
test_probas2, test_targets2, test_preds2 = learn.get_X_preds(X_test, y_test, with_decoded=True)
test_probas2, test_targets2, test_preds2

(TSTensor(vars:30, len:4, device=cpu),
 TensorCategory([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3]),
 array(['4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4'], dtype='<U1'))

In [ ]:
test_probas3, test_targets3, test_preds3, test_losses3 = learn.get_X_preds(X_test, y_test, with_loss=True, with_decoded=True)
test_probas3, test_targets3, test_preds3, test_losses3

(TSTensor(vars:30, len:4, device=cpu),
 TensorCategory([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3]),
 array(['4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4', '4',
        '4', '4', '4', '4'], dtype='<U1'),
 TensorBase([1.3874, 1.3874, 1.3874, 1.3874, 1.3874, 1.3708, 1.3708, 1.3708, 1.3708,
         1.3708, 1.3708, 1.3708, 1.3708, 1.3708, 1.4503, 1.4504, 1.4504, 1.4504,
         1.3399, 1.3399, 1.3399, 1.3399, 1.3399, 1.3399, 1.3399, 1.3399, 1.3399,
         1.3399, 1.3398, 1.3399]))

In [ ]:
from fastcore.test import test_eq
test_eq(test_probas, test_probas2)
test_eq(test_preds, test_preds2)
test_eq(test_probas, test_probas3)
test_eq(test_preds, test_preds3)

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);